## Learning ipywidgets: a very simple d3 "CircleWidget" with a js CircleView 


I use this notebook to get acquainted with the mechanics and best practices for coding interactive ipywidgets, for Jupyter notebooks and the forthcoming Jupyter lab.    If successful, it may be useful to others who have already worked through the excellent [tutorial and 'datepicker' examples](https://ipywidgets.readthedocs.io/en/latest/) provided by the ipywidgets team.

Once this simple notebook, illustrating the use of a "foreign" graphics library with ipywidgets, has passed muster, I will add documentation on the extra steps to produce a proper cookiecutter nbextensions template, and eventually into jupyter lab.

Specify the location of the d3 library:

In [74]:
%%javascript
require.config({
    paths: {
        d3: '//cdnjs.cloudflare.com/ajax/libs/d3/3.4.8/d3.min'
    },
});


<IPython.core.display.Javascript object>

In [75]:
import ipywidgets as widgets
from traitlets import Int, Unicode, List, CInt, Dict, validate

class CircleWidget(widgets.DOMWidget):
    _view_name = Unicode('CircleView').tag(sync=True)
    _view_module = Unicode('circle').tag(sync=True)
    radius = Int(100).tag(sync=True)
    circles = List(trait=Int(19)).tag(sync=True)
    circleDict = Dict().tag(sync=True)
    def drawCircle(self, newRadius):
       self.radius = newRadius
       #self.circles = self.circles.append(newRadius)
       self.circles = [newRadius, newRadius-1]
       self.circleDict["aa"] = newRadius


One obvious flaw below is the preservation of WidgetView state via the browser "window" object.
My attempts to use these variables failed, with assignment to them in one method not persisting:

    this.svgCreated
    this.svg
    
 I will be grateful for insight on how to do that, or otherwise (without globals) preserve state.

<b>DOM latency</b>:  I discovered that the div (to which the d3 svg gets added) does not appear instantly in the DOM.
The current solution is to delay 500 msecs before calling <i>createCanvas</i>.  A more robust approach would wait for the actual appearance of that underlying div in the DOM.  Suggestions?

In [77]:
%%javascript

require.undef('circle');

define('circle', ["jupyter-js-widgets", "d3"], function(widgets, d3) {
    
    var CircleView = widgets.DOMWidgetView.extend({

        initialize: function() {
           window.svgCreated = false;
           window.svg = null;
           },

        createDiv: function(){
            var div = $("<div id='d3DemoDiv' style='border:1px solid red; height: 300px; width: 600px'>")
            return(div);
            },
 
        createCanvas: function(){
           var svg = d3.select("#d3DemoDiv")
                       .append("svg")
                       .attr("id", "svg").attr("width", 600).attr("height", 300);
           window.svgCreated = true;
           window.svg = svg;
           }, // createCanvas

        drawCircle: function(radius){
           window.svg.append("circle")
              .style("stroke", "gray")
              .style("fill", "white")
              .attr("r", radius)
              .attr("cx", 300)
              .attr("cy", 150)
              .on("mouseover", function(){d3.select(this).style("fill", "aliceblue");})
              .on("mouseout",  function(){d3.select(this).style("fill", "white");});
            },

        render: function() { 
            this.$el.append(this.createDiv());
            this.listenTo(this.model, 'change:radius',  this._radius_changed,  this);
            this.listenTo(this.model, 'change:circles', this._circles_changed, this);
            this.listenTo(this.model, 'change:circleDict', this._circleDict_changed, this);
            setTimeout(this.createCanvas, 500);
            },

        _radius_changed: function() {
           var newRadius = this.model.get('radius');
           this.drawCircle(newRadius);
           console.log("radius_changed");
           },

        _circles_changed: function() {
           console.log("_circles_changed")
           console.log(this.model.get('circles'));
           },

        _circleDict_changed: function() {
           console.log("_circleDict_changed")
           console.log(this.model.get('circleDict'));
           }

    });
    return {
        CircleView : CircleView
    };
});

<IPython.core.display.Javascript object>

How to add extra args to the CircleWidget ctor, pass them into the CircleView js object?

In [78]:
cw = CircleWidget(height=300, width=500)  # extra args currently ignored.  how to parse and send to javascript?
cw

Modifiying the traitlet Int radius field triggers radius_changed, circle is drawn with new radius

In [79]:
cw.drawCircle(10)

In [80]:
cw.radius = 80
cw.radius = 30

In [81]:
cw.circles

[10, 9]

In [82]:
cw.drawCircle(88)

How does one define a (python) function in a DomWidget subclass, of this general sort?

    cw.drawCircle(radius=30, x=100, y=100, borderColor="red", fillColor="white")
    

In [73]:
cw.circles

[88, 87]

In [10]:
cw.drawCircle(33)

In [83]:
cw.circleDict

{'aa': 88}